In [ ]:
import numpy as np
import mygrad as mg
from mygrad import Tensor

from noggin import create_plot
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from mynn.layers.dense import dense
from mygrad.nnet.initializers import glorot_uniform

In [ ]:
class Model:

    def __init__(self, D_input=512, D_glove=200):
        self.dense1 = dense(D_input, D_glove, weight_initializer=glorot_normal) 

    def __call__(self, x):
        return self.dense1(x)

    @property
    def parameters(self):
        return self.dense1.parameters

In [ ]:
from mynn.optimizers.sgd import SGD
from mygrad.nnet.losses import margin_ranking_loss

optim = SGD(model.parameters, learning_rate=1e-3, momentum=0.9)

In [1]:
# (1,200)       (1,512)          (1,512)

# [caption1, confusion_image, truth_image]
# |caption2, confusion_image, truth_image|
# [caption3, confusion_image, truth_image]

SyntaxError: invalid syntax (4090296465.py, line 2)

In [ ]:
def accuracy(prediction_truth, prediction_confusion, truth_caption):
    truth_count = np.count_nonzero(np.dot(prediction_truth, truth_caption) > np.dot(prediction_confusion, truth_caption))
    return truth_count / len(truth_caption)

In [ ]:
batch_size = 32
num_epochs = 1

model = Model(512, 200)

for epoch_cnt in range(num_epochs):
    idxs = np.arange(len(train)) 
    np.random.shuffle(idxs)

    for batch_cnt in range(len(train) // batch_size):
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]
        
        # captions
        truth_caption_ID = train[batch_indices][0]
        truth_caption = truth_caption[truth_caption_ID]
        #image and confusion
        truth_image = train[batch_indices][1]
        confusion = train[batch_indices][2]
        
        prediction_truth = model(truth_image)  
        prediction_confusion = model(confusion)
        
        sim_truth = np.dot(prediction_truth, truth_caption)
        sim_confusion =  np.dot(prediction_confusion, truth_caption)
        
        # add einsum 
        loss = margin_ranking_loss(sim_truth, sim_confusion, truth_caption, margin=0.25) 

        loss.backward() 

        optim.step() 

        acc = accuracy(prediction_truth, prediction_confusion, truth_caption)
        
        plotter.set_train_batch(
            {"loss": loss.item(), "accuracy": acc}, batch_size=batch_size
        )
        
    for batch_cnt in range(0, len(x_test) // batch_size):
        idxs = np.arange(len(x_test))
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size] 
        batch = x_test[batch_indices]

        with mg.no_autodiff:
            # captions
            truth_caption_ID = test[batch_indices][0]
            truth_caption = truth_caption[truth_caption_ID]
            #image and confusion
            truth_image = test[batch_indices][1]
            confusion = test[batch_indices][2]

            prediction_truth = model(truth_image)  
            prediction_confusion = model(confusion)
            
            acc = accuracy(prediction_truth, prediction_confusion, truth_caption)

        plotter.set_test_batch({"accuracy": acc}, batch_size=batch_size)

    plotter.set_train_epoch()
    plotter.set_test_epoch()
plotter.plot()